# Programmieren in Python
## Lambda-Ausdrücke, Listenabstraktion, Generatoren, Dekoratuere

 ---

## 1. Lambda-Ausdrücke

Syntax:
```python
lambda *arguments: expression
```

Entspricht der benannten Funktion
```python
def func(*arguments):
    return expression
```

Beispiel mit der Funktion    $~ f:\mathbb{R}\rightarrow\mathbb{R},~ x \mapsto x^2$

Ohne Lambda:

In [ ]:
def f(x : float) -> float:
    return x ** 2

Nun mit Lambdas:

In [ ]:
f = lambda x: x ** 2

* Vorteile:

    - weniger Keywords um einfache Funktionen zu schreiben
    - benötigt keinen Namen
    - Anwendung für Mappings, Filter usw.

* Nachteile:
    - keine Type-Annotations möglich (ab Python 3.6 möglich)
    - für mehr als simple Funktionen unübersichtlich

In [ ]:
# Beispiel für Anwendung

data = [1,2,3,4,5,6,7,8,9,10] # some data

divisible_by_three = filter(lambda x: x % 3 == 0, data)

print(list(divisible_by_three))

---

## 2. Comprehensions

In Python können Listen, Sets, und Dictionaries mit einfacher Syntax generiert werden

Beispiel:

    a_list = [False, False, True, False, False]
    a_set  = {1, 4, 9, 16, 25}
    a_dict = {'a': 1, 'aa': 2, 'aaa': 3, 'aaaa': 4}
    
Wollen wir Listen, Sets und Dictionaries nach einer bestimmten Regel generieren kann man dies in einer Schleife tun:
```python
a_list = []
for i in range(5):
    a_list.append(i % 3 == 2)

a_set = set()
for i in range(1, 6):
    a_set.add(i ** 2)

a_dict = {}
for i in range(1, 5):
    a_dict['a'*i] = i 
```    
Python hat dafür eine einfachere schreibweise, die nicht nur verständlicher, sondern auch schneller ist $-$ die Comprehensions
Die List-, Set- und Dictionary-Comprehensions sind wie folgt aufgebaut:

```python
# Vereinfacht
liste_a = [ expression for target_list in smth ]
```
Man kann mehr als eine for-Schleife benutzen
```python
liste_b = [ expression for target_a in smth1 for target_b in smth2 ]
```

Und auch Tests sind möglich
```python
liste_c = [ expression for target in smth if condition ] 
```

Am besten einfach ein paar Beispiele

```python
# Die drei Beispiele von gerade eben mit Comprehensions

a_list = [i % 3 == 2 for i in range(5)]
a_set  = {i ** 2 for i in range(1,6)}
a_dict = {i*'a': i for i in range(1,6)}


""" 
* Wenn man mit einer 2D-Liste arbeitet benötigt man manchmal die 8 Nachbarn
* einer Zelle (z.B. Conways Game of Life, Minesweeper)
* 
"""
# array : List[List[int]]
# pos_x : int
# pos_y : int
neighbors = [array[pos_x+xoff][pos_y+yoff] for xoff in {-1,0,1}
                                           for yoff in {-1,0,1}
                                           if xoff != or yoff != 0 ]


"""
* 
"""

```
---

## 3. Generatoren

Für den Einstieg in Generatoren schauen wir uns diese Grafik an:

<figure>
   <img src="http://nvie.com/img/relationships.png" width="600" pady="50" title="Quelle: http://nvie.com/img/relationships.png">
  <figcaption> Quelle: http://nvie.com/img/relationships.png </figcaption>
</figure>



Definition aus dem Source Code von collections.abc
Zeigt die Abstract Base Classes von Iterable, Iterator und Generator:

(Implementation der Funktionen fehlt, soll nur zeigen, welche Methoden implementiert werden müssen)
```python
from abc import ABCMeta, abstractmethod


class Iterable(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __iter__(self):
        return NotImplemented

class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        return NotImplemented

    def __iter__(self):
        return self

    
class Generator(Iterator):

    __slots__ = ()

    def __next__(self):
        """Return the next item from the generator.
        When exhausted, raise StopIteration.
        """
        return NotImplemented

    @abstractmethod
    def send(self, value):
        """Send a value into the generator.
        Return next yielded value or raise StopIteration.
        """
        return NotImplemented

    @abstractmethod
    def throw(self, typ, val=None, tb=None):
        """Raise an exception in the generator.
        Return next yielded value or raise StopIteration.
        """
        return NotImplemented

    def close(self):
        """Raise GeneratorExit inside generator.
        """
        try:
            self.throw(GeneratorExit)
        except (GeneratorExit, StopIteration):
            pass
        else:
            raise RuntimeError("generator ignored GeneratorExit")

```

Generatoren können auch eleganter geschrieben werden. Eine Möglichkeit ist die Generator-Expression. Sie ist ähnlich zu den Comprehensions jedoch gibt sie ein Generator Objekt zurück.

In [ ]:
zaehlen = (i for i in range(10))

In [ ]:
print(type(zaehlen))
try:
    print(next(zaehlen))
except StopIteration:
    print("Ende")

Möchte man komplexere Generatoren schreiben hilft einem die Generator Funktion. Sie sieht ähnlich zu normalen Funktionen aus, jedoch benutzt sie das Keyword ```yield``` an Stelle von ```return```. Bei jedem Aufruf von ```next``` oder ```.send```, arbeitet der Generator dann bis zum Keyword ```yield``` , gibt das Element aus und pausiert dann bis zum nächsten Aufruf von ```next``` oder ```.send```

In [ ]:
def gen():
    for i in range(10):
        yield i
generator = gen()

In [ ]:
print(type(generator))
try:
    print(next(generator))
except StopIteration:
    print("Ende")

Wenn die Methode ```.send(value)``` benutzt wird, nimmt das yield statement den Wert von ```value``` an.

```python
>>> def send_test():
        eins = yield "1"
        zwei = yield "2"
        drei = yield (eins, zwei)
        yield drei
    
>>> test = send_test()
>>> next(test)          # same as test.send(None)
"1"
>>> test.send("Hello")
"2"
>>> test.send("World")
("Hello", "World")
>>> test.send("Test")
"Test"
>>> test.send("Jetzt ist Schluss")
StopIteration
```

Generatoren müssen kein letzes Element haben. Es ist absolut erlaubt so etwas zu schreiben:

```python
def numbers(start=0, step=1):
    while True:
        yield start
        start += step
```
Dies ist sogar so oft genutzt, dass es einen Iterator im ```itertools``` Modul von python gibt, (fast) das gleiche Ergebnis erzielt. (itertools.count(start=0,step=0))
```python

```

In [81]:
from functools import reduce
list(reduce(lambda a, b: [*a, a[-1]+b], [12, -1, 7],[0]))

[0, 12, 11, 18]

In [82]:
a = []


In [83]:
a = [1,2,3]
a.pop()

3

In [86]:
a.pop()

IndexError: pop from empty list

In [88]:
b = iter({1,2,3})

In [93]:
{ a: len(a) for a in ['aaa', 'a', 'abcd'] }

{'aaa': 3, 'a': 1, 'abcd': 4}

In [95]:
x = 0
y = 0
x == y == 0

True

In [96]:
sum( [ x for x in range(10) ] )

45

In [98]:
a = iter( [ x for x in range(10) ] )

In [99]:
 b = ( x  for x in range(10) )
from itertools import i

In [101]:
all( x == y for x, y in zip(a,b) )

True